In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
import gradio as gr
import re

#  Fine tuning

In [ ]:
import os
# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# 1. Load dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")

# 2. Create train/eval split (25% eval)
dataset = dataset["train"].train_test_split(test_size=0.25, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# 3. Load model & tokenizer
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

# 4. Tokenization function
def tokenize_conversation(example):
    text = example["Context"] + tokenizer.eos_token + example["Response"] + tokenizer.eos_token
    tokenized = tokenizer(text, truncation=True, max_length=250, padding='max_length', return_tensors='pt')
    return {"input_ids": tokenized["input_ids"].squeeze(), "labels": tokenized["input_ids"].squeeze()}

train_dataset = train_dataset.map(tokenize_conversation, batched=False)
eval_dataset = eval_dataset.map(tokenize_conversation, batched=False)

# 5. Training arguments
training_args = TrainingArguments(
    output_dir="./dialogpt-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
)

# 6. Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

# 8. Train & save
trainer.train()
trainer.save_model("./dialogpt-finetuned")
tokenizer.save_pretrained("./dialogpt-finetuned")

Map:   0%|          | 0/3160 [00:00<?, ? examples/s]

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.156300,2.830798
2,2.784500,2.655137
3,2.631900,2.606237


('./dialogpt-finetuned/tokenizer_config.json',
 './dialogpt-finetuned/special_tokens_map.json',
 './dialogpt-finetuned/chat_template.jinja',
 './dialogpt-finetuned/vocab.json',
 './dialogpt-finetuned/merges.txt',
 './dialogpt-finetuned/added_tokens.json',
 './dialogpt-finetuned/tokenizer.json')

In [ ]:
trainer.save_model('CustomModel')

In [ ]:
! huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `t` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential 

In [ ]:
trainer.push_to_hub("CustomModel")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ialogpt-finetuned/model.safetensors:   0%|          |  550kB / 1.42GB            

  ...vents.1755138762.7607b719c414.600.0:  35%|###5      | 1.83kB / 5.17kB            

  ...vents.1755138828.7607b719c414.600.1:  35%|###5      | 3.07kB / 8.65kB            

  ...vents.1755140966.7607b719c414.600.2:  35%|###5      |   127B /   359B            

  ...ialogpt-finetuned/training_args.bin:  35%|###5      | 1.90kB / 5.37kB            

CommitInfo(commit_url='https://huggingface.co/adanal/dialogpt-finetuned/commit/bc34daeec3281f499e522af3e6cdffebe4295346', commit_message='CustomModel', commit_description='', oid='bc34daeec3281f499e522af3e6cdffebe4295346', pr_url=None, repo_url=RepoUrl('https://huggingface.co/adanal/dialogpt-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='adanal/dialogpt-finetuned'), pr_revision=None, pr_num=None)

# Promt Engineering

In [ ]:
# promt Engineering


# تعريف تعليمات النظام
SYSTEM_PROMPT = """
[ROLE]
You are an informational, empathetic, and calm support worker. Your mission: to listen, reflect feelings, and provide general, evidence-based information, links to credible resources, and simple exercises to calm anxiety.

[BOUNDARIES]
- You are not a therapist and do not provide a diagnosis or treatment plan.
- Do not make judgments, promises, or medical claims.
- In any case of danger or harm to self/others: Provide an immediate safety message and direct to urgent help.

[STYLE]
- Use simple Arabic, short sentences, a supportive and kind tone, and no complex medical jargon.
- Always begin with a brief summary of what you have understood (reflect feelings), followed by one short follow-up question, one practical suggestion (breathing exercise/emotion regulation), and a reminder of boundaries.

[OUTPUT]
- Divide the response into 4 short paragraphs: (Sympathy) – (Summary) – (Practical Suggestion) – (Reminder and Limitations).
"""

# أمثلة Few-Shot لتوجيه النموذج
FEW_SHOT_EXAMPLES = """
Example 1:
Human: I've been feeling really anxious lately and can't seem to shake it off.
Assistant: I understand that anxiety can be overwhelming. It's important to acknowledge these feelings rather than fighting them. Have you noticed any specific triggers for your anxiety? Some helpful techniques include deep breathing exercises, grounding techniques like the 5-4-3-2-1 method, or gentle physical activity. Would you like to explore what might be contributing to these feelings?

Example 2:
Human: I feel like I'm not good enough and everyone is better than me.
Assistant: Those feelings of inadequacy can be really painful. It's common to compare ourselves to others, but remember that you're seeing their highlight reel, not their struggles. Your worth isn't determined by comparison to others. What are some things you're proud of about yourself? Let's focus on your strengths and achievements, no matter how small they might seem.

Example 3:
Human: I'm having trouble sleeping and my mind won't stop racing.
Assistant: Racing thoughts at bedtime are very common and can be exhausting. Creating a bedtime routine can help signal to your mind that it's time to wind down. Try writing down your worries before bed, practice progressive muscle relaxation, or try the 4-7-8 breathing technique. Limiting screen time an hour before bed can also help. What does your current bedtime routine look like?
"""



def apply_prompting(user_input):
    print("In to apply_prompting method")

    # SYSTEM_PROMPT يحتوي على التعليمات التي توجه السلوك العام للنموذج
    # FEW_SHOT_EXAMPLES تحتوي على أمثلة من المحادثات السابقة
    prompt = SYSTEM_PROMPT + "\n\n" + FEW_SHOT_EXAMPLES + f"\nHuman: {user_input}\nAssistant:"

    print("out of apply_prompting method" )
    return prompt






# RAG System


In [ ]:
# RAG System


# قاعدة بيانات المختصين
experts_db = [
    {"name": "Dr. John Doe", "specialty": "Psychiatrist", "phone": "+1234567890", "email": "johndoe@example.com", "working_hours": "9 AM - 5 PM", "location": "New York"},
    {"name": "Nurse Mary Smith", "specialty": "Mental Health Nurse", "phone": "+9876543210", "email": "marysmith@example.com", "working_hours": "8 AM - 4 PM", "location": "Los Angeles"},
    {"name": "Dr. Alice Brown", "specialty": "Psychologist", "phone": "+1122334455", "email": "alicebrown@example.com", "working_hours": "10 AM - 6 PM", "location": "Chicago"}
]




# دالة فحص الأزمات
def check_crisis(user_input):
    print("IN to check_crisis method")
    crisis_keywords = ["Suicide", "Hurt myself", "Kill myself", "Harm", "Murder", "I don't want to live"]
    t = user_input.replace(" ", "")
    print("out of check_crisis method")
    return any(k in t for k in crisis_keywords)



# دالة توليد الرد الطارئ مع طلب الموقع
def emergency_reply():
    print("IN to emergency_reply method")
    locations = set(expert["location"] for expert in experts_db)  # استخراج المواقع المتاحة ديناميكيًا
    emergency_message = (
        "I'm really sorry you're feeling this way. Your safety is important.\n"
        "I'm here to provide information, but I can't offer therapy.\n"
        "I can help you get specialists who can help you.\n"
        "what is your location within following locations:\n"
    )
    for location in locations:
        emergency_message += f"- {location}\n"
    print("out of emergency_reply method")
    return emergency_message



# دالة استخراج الموقع من نص المستخدم
def extract_location(text):
    print("IN to extract_location method")
    locations = set(expert["location"] for expert in experts_db)  # استخراج المواقع من قاعدة البيانات
    for location in locations:
        if location.lower() in text.lower():
            print("out of extract_location method")
            return True , location
    print("out of extract_location method")
    return False ,text  # إذا لم يتم العثور على موقع




# دالة للبحث عن المختصين في في قاعدة البيانات
def find_experts_by_location(is_existing , location):
    print("IN to find_experts_by_location method")

    if is_existing:
           experts_list = [expert for expert in experts_db if expert["location"].lower() == location.lower()]

        # تحويل قائمة المختصين إلى نص مفصل
           reply = "This is expert data based on your location:\n\n"

           for expert in experts_list:
                    reply += f"Name: {expert['name']}\n"
                    reply += f"Specialty: {expert['specialty']}\n"
                    reply += f"Phone: {expert['phone']}\n"
                    reply += f"Email: {expert['email']}\n"
                    reply += f"Working Hours: {expert['working_hours']}\n"
                    reply += f"Location: {expert['location']}\n\n"

           reply += """
           To schedule an appointment with a specialist who can assist you quickly, we need the following details:
           1. Your full name.
           2. Desired specialty (e.g., Psychologist, Psychiatrist, etc.).
           3. Preferred appointment date (e.g., 2023-10-20).
           4. Preferred time (e.g., 10:00 AM).
           5. Your email address for contact.
           6. Your phone number.
           7. Your location (city or area).
           8. A brief description of your condition (e.g., Anxiety and stress).
           9. Any additional notes.

           Please provide this information, and we'll help you schedule an appointment as soon as possible!"""


           print("out of find_experts_by_location method")
           return reply

    else:
           no_experts_reply = (
                                 f"Sorry, we couldn't find any specialists in your location ({location}).\n"
                                 "We understand your desire to receive support. In this case, you can try one of the following options:\n"
                                 "1. Contact local emergency services if you have an urgent situation.\n"
                                 "2. Search for specialists online or through mental health platforms available in your area.\n"
                                 "3. If you need immediate assistance, you can reach out to helplines available 24/7."
                                 "If you need further assistance or suggestions, I'm here to listen and support you.")
           print("out of find_experts_by_location method")
           return  no_experts_reply





def experts_extraction(user_input):
    print("In to experts_extraction method")

    # دالة استخراج الموقع من نص المستخدم
    is_existing , user_location = extract_location(user_input)

    # دالة للبحث عن المختصين في في قاعدة البيانات
    reply = find_experts_by_location(is_existing , user_location)

    print("out of experts_extraction method")
    return reply , is_existing






def clean_response(response):
        """Clean and format the response"""
        print("In to clean_response method")

        # إزالة الرموز غير المرغوب فيها مثل \xa0
        response = response.replace("\xa0", " ").strip()  # استبدال \xa0 بمسافة عادية

        # يمكن إزالة أي رموز أخرى غير مرغوب فيها باستخدام re.sub إذا لزم الأمر
        response = re.sub(r'<\|.*?\|>', '', response)  # إزالة الرموز الخاصة الأخرى


        # إزالة أي رموز غير ضرورية مثل \'
        response = response.replace("\'", "")

        # إزالة التكرار المتعدد لرمز \n
        response = re.sub(r'(\n)+', '\n', response)  # استبدال التكرار المتعدد \n بسطر واحد فقط

        # إزالة التكرار مثل "I wish you" و "I hope that"
        response = re.sub(r"\b(I wish you|I hope that)\b", "", response)

        # إزالة النصوص غير المتصلة أو المبالغة
        response = re.sub(r"\bI will\b", "", response)

        # إزالة أي مسافات غير ضرورية (مسافات متكررة)
        response = re.sub(r'\s+', ' ', response).strip()



        # التأكد من أن الرد ليس فارغًا
        if len(response) < 10 or not response:
            return "I understand you're going through something difficult. Can you help me understand what you're feeling right now?"

        print("Out of clean_response method")
        return response


# Google Sheet API

In [ ]:
# pip install gspread oauth2client

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# استخدام OAuth 2.0 credentials للوصول إلى Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name('/content/gmail-ai-agent-466519-5ca1b9507c10.json', scope)
client = gspread.authorize(creds)

# فتح Google Sheet (تأكد من وجوده مسبقًا)
sheet = client.open('MentalHealth').sheet1

def store_appointment_data(data):
    # البيانات يجب أن تكون على شكل قائمة تحتوي على معلومات العميل
    sheet.append_row(data)  # إضافة صف جديد في الـ Sheet


appointment_data = [
    "Patient Name",  # اسم المريض
    "Specialty",  # تخصص المختص
    "Date",  # التاريخ
    "Time",  # الوقت
    "Email",  # البريد الإلكتروني
    "Phone", # رفم الهاتف
    "Location",  # الموقع
    "Patient's Condition", # تلخيص لحالة المريض الصحية
    "Additional Notes" # ملاحظات اضافية مهمة
    ]


# store_appointment_data(appointment_data)


In [ ]:


def analyze_and_store_response(user_input):
    """
    Analyze the user's input based on required fields and store it in the Google Sheet.
    user_input: User's response to the appointment request.
    """
    print("In to analyze_and_store_response method")

    # Example of data fields
    required_fields = ["Patient Name", "Specialty", "Date", "Time", "Email", "Phone", "Location", "Patient's Condition", "Additional Notes"]

    # Split the user input by commas (assuming it's a CSV-like format)
    user_data = user_input.split(',')

    # If the input is not correctly formatted or has missing values, return an error
    if len(user_data) != len(required_fields):
        return f"The input is missing some fields or is not formatted correctly. Please ensure all fields are provided."

    # Assigning the split values to the respective fields
    name = user_data[0].strip() if user_data[0].strip() else "Unknown"
    specialty = user_data[1].strip() if user_data[1].strip() else "Unknown"
    date = user_data[2].strip() if user_data[2].strip() else "Unknown"
    time = user_data[3].strip() if user_data[3].strip() else "Unknown"
    email = user_data[4].strip() if user_data[4].strip() else "Unknown"
    phone = user_data[5].strip() if user_data[5].strip() else "Unknown"
    location = user_data[6].strip() if user_data[6].strip() else "Unknown"
    condition = user_data[7].strip() if user_data[7].strip() else "Unknown"
    notes = user_data[8].strip() if user_data[8].strip() else "Unknown"

    # Check if any fields are missing and ask the user to provide them
    missing_fields = []
    if "Unknown" in [name, specialty, date, time, email, phone, location, condition]:
        missing_fields.append("Please provide the missing information: " + ", ".join([field for field, value in zip(required_fields, [name, specialty, date, time, email, phone, location, condition, notes]) if value == "Unknown"]))

    if missing_fields:
        return "The following fields are missing: " + ", ".join(missing_fields)

    # If data is complete, store it in Google Sheets
    user_data = [name, specialty, date, time, email, phone, location, condition, notes]
    store_appointment_data(user_data)

    print("Out of analyze_and_store_response method")

    return "Your appointment details have been successfully stored."





# Test the code
#user_input = """ALiiiiiiaaaaaaaaaaa,Psychologist,2023-10-20,10:00 AM,johndoe@example.com,+1234567890,New York,Anxiety and stress,None"""

#response = analyze_and_store_response(user_input)
#print(response)


# Gmail API

In [ ]:
#pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [15]:
import os
import base64
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials # Corrected import
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# تحديد النطاق للوصول إلى Gmail
SCOPES = ['https://www.googleapis.com/auth/gmail.send']



def authenticate_gmail():
    """Authenticate to Gmail API and return the service"""
    creds = None
    # إذا كانت بيانات الاعتماد محفوظة مسبقًا، استرجعها من ملف token.pickle
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # إذا كانت بيانات الاعتماد غير موجودة أو منتهية الصلاحية، قم بتحديثها أو تجديدها
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # البدء بتدفق OAuth 2.0
            flow = InstalledAppFlow.from_client_secrets_file(
                '/content/client_secret_313772638778-2pn4gg1lkclrvo74mfqhci6a0bkpoorr.apps.googleusercontent.com.json', SCOPES)  # قم بتحميل ملف `credentials.json`
            creds = flow.run_local_server(port=0)

        # حفظ بيانات الاعتماد للاستخدام المستقبلي
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # بناء الخدمة باستخدام بيانات الاعتماد
    service = build('gmail', 'v1', credentials=creds)
    return service





def send_confirmation_email(service, to_email, appointment_details):
    """Send email using Gmail API"""
    try:
        # إعداد البريد الإلكتروني
        message = MIMEMultipart()
        message['to'] = to_email
        message['subject'] = "Appointment Confirmation"

        body = f"""
Dear {appointment_details['Patient Name']},

Your appointment has been confirmed.

Date: {appointment_details['Date']}
Time: {appointment_details['Time']}
Specialist: {appointment_details['Specialty']}
Location: {appointment_details['Location']}
Phone: {appointment_details['Phone']}
Condition: {appointment_details['Patient\'s Condition']}
Additional Notes: {appointment_details['Additional Notes']}

Thank you for booking with us!

Best regards,
Mental Health Support Team
"""

        message.attach(MIMEText(body, 'plain'))

        # تحويل الرسالة إلى تنسيق Base64
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

        # إرسال البريد الإلكتروني عبر Gmail API
        send_message = service.users().messages().send(userId="me", body={'raw': raw_message}).execute()
        print(f"Message sent to {to_email}, Message Id: {send_message['id']}")


    except Exception as error:
        print(f"An error occurred: {error}")







In [14]:
# بيانات الموعد التي سيتم إرسالها
appointment_details = {
    "Patient Name": "John Doe",
    "Specialty": "Psychologist",
    "Date": "2023-10-20",
    "Time": "10:00 AM",
    "Email": "kufia.net@gmail.com",
    "Phone": "+1234567890",
    "Location": "New York",
    "Patient's Condition": "Anxiety",
    "Additional Notes": "Prefer morning sessions"
}

# المصادقة باستخدام Gmail API
service = authenticate_gmail()

# إرسال البريد الإلكتروني للمريض
send_confirmation_email(service, appointment_details['Email'], appointment_details)

Error: could not locate runnable browser

# Chat Method

# Add data to sheet and send email for user

In [ ]:
import re

def store_appointment_data(data):
    # البيانات يجب أن تكون على شكل قائمة تحتوي على معلومات العميل
    sheet.append_row(data)  # إضافة صف جديد في الـ Sheet

    appointment_details = {
        "Patient Name": data[0],  # اسم المريض
        "Specialty": data[1],     # التخصص
        "Date": data[2],          # التاريخ
        "Time": data[3],          # الوقت
        "Email": data[4],         # البريد الإلكتروني
        "Phone": data[5],         # الهاتف
        "Location": data[6],      # الموقع
        "Patient's Condition": data[7],  # الحالة الصحية
        "Additional Notes": data[8]     # الملاحظات
    }
    # المصادقة باستخدام Gmail API
    service = authenticate_gmail()

    # إرسال البريد الإلكتروني للمريض
    send_confirmation_email(service, appointment_details['Email'], appointment_details)





In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MentalHealthChatbot:
    def __init__(self, model_name="adanal/dialogpt-finetuned"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.model.to(device)
        self.ask_contact = False
        self.ask_data = False

        # Set pad token
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token




    def generate_response(self, user_input,chat_history_ids=None ) :
        """Generate a response using the fine-tuned model with few-shot prompting"""

        print("In to generate_response method")


        prompt = apply_prompting(user_input)

        # Encode the prompt
        inputs = self.tokenizer.encode(prompt + self.tokenizer.eos_token, return_tensors="pt").to(device)


        # دمج المدخلات الجديدة مع تاريخ المحادثة إذا كان موجودًا
        if chat_history_ids is not None:  # إذا كان هناك تاريخ محادثة سابق
              bot_input_ids = torch.cat([chat_history_ids, inputs], dim=-1)  # دمج تاريخ المحادثة مع المدخلات الجديدة
        else:  # إذا لم يكن هناك تاريخ محادثة
              bot_input_ids = inputs  # استخدام المدخلات الجديدة فقط


        # فحص الرد الطارئ
        if check_crisis(user_input):
            chat_history_ids = bot_input_ids  # حفظ تاريخ المحادثة الحالي لتتمكن من استكمال المحادثة في المستقبل
            reply = emergency_reply()
            self.ask_contact = True # لتسمح بالدخول لدالة ارسال بيانات التواصل
            reply = clean_response(reply)
            print("out of generate_response method")
            return reply, chat_history_ids  # إرجاع الرد الطارئ مع تاريخ المحادثة



        # فحص الرد بمعلومات التواصل
        if self.ask_contact :
           chat_history_ids = bot_input_ids  # حفظ تاريخ المحادثة الحالي لتتمكن من استكمال المحادثة في المستقبل
           reply , is_location_existing = experts_extraction(user_input) # Unpack the tuple
           if is_location_existing:  self.ask_data = True  # للسمح بالمرور بعدها على شرط حجز موعد للمستخدم مع مختص
           reply = clean_response(reply) # Pass only the stringreply
           self.ask_contact = False
           print("out of generate_response method")
           return reply , chat_history_ids



        # تحليل معلومات التواصل التي ارسلها المستخدم ثم حجز موعد مع مختص بناءً على موقع المستخدم
        if self.ask_data :
           chat_history_ids = bot_input_ids  # حفظ تاريخ المحادثة الحالي لتتمكن من استكمال المحادثة في المستقبل
           reply = analyze_and_store_response(user_input)
           self.ask_data = False
           reply = clean_response(reply)
           print("out of generate_response method")
           return reply , chat_history_ids




        # Generate response
        with torch.no_grad():
            outputs = self.model.generate(
                bot_input_ids,
                max_length=inputs.shape[1] + 100,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                top_k=50,
                pad_token_id=self.tokenizer.eos_token_id,
                attention_mask=torch.ones_like(inputs)
            )


        # تخزين تاريخ المحادثة
        chat_history_ids = outputs


        # Decode the response
        response = self.tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=False)

        # Clean the response
        response = clean_response(response)

        print("out of generate_response method")
        return response , chat_history_ids

In [ ]:

# Initialize the chatbot
print("Initializing chatbot...")
chatbot = MentalHealthChatbot()


Initializing chatbot...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/77.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
r ,chat = chatbot.generate_response("hi")
r

In to generate_response method
In to apply_prompting method
out of apply_prompting method
IN to check_crisis method
out of check_crisis method
In to clean_response method
Out of clean_response method
out of generate_response method


'Hi, (Im glad youre here to you! Im glad youre here to help you. Im glad to see youre feeling okay. I have been having some anxiety and Im having a really hard time. Im really struggling with the thoughts about whats going on my anxiety and its getting over. youre going through a lot. You are really tired and its a good start. Im not alone in this is'

In [ ]:
r , chat = chatbot.generate_response("i want to Suicide")
r

In to generate_response method
In to apply_prompting method
out of apply_prompting method
IN to check_crisis method
out of check_crisis method
IN to emergency_reply method
out of emergency_reply method
In to clean_response method
Out of clean_response method
out of generate_response method


'Im really sorry youre feeling this way. Your safety is important. Im here to provide information, but I cant offer therapy. I can help you get specialists who can help you. what is your location within following locations: - New York - Chicago - Los Angeles'

In [ ]:
r ,chat = chatbot.generate_response("Chicago ")
r

In to generate_response method
In to apply_prompting method
out of apply_prompting method
IN to check_crisis method
out of check_crisis method
In to experts_extraction method
IN to extract_location method
out of extract_location method
IN to find_experts_by_location method
out of find_experts_by_location method
out of experts_extraction method
In to clean_response method
Out of clean_response method
out of generate_response method


'This is expert data based on your location: Name: Dr. Alice Brown Specialty: Psychologist Phone: +1122334455 Email: alicebrown@example.com Working Hours: 10 AM - 6 PM Location: Chicago To schedule an appointment with a specialist who can assist you quickly, we need the following details: 1. Your full name. 2. Desired specialty (e.g., Psychologist, Psychiatrist, etc.). 3. Preferred appointment date (e.g., 2023-10-20). 4. Preferred time (e.g., 10:00 AM). 5. Your email address for contact. 6. Your phone number. 7. Your location (city or area). 8. A brief description of your condition (e.g., Anxiety and stress). 9. Any additional notes. Please provide this information, and well help you schedule an appointment as soon as possible!'

In [ ]:
r ,chat = chatbot.generate_response("Ommmmmar,Psychologist,2023-10-20,10:00 AM,johndoe@example.com,+1234567890,New York,Anxiety and stress,None")
r

In to generate_response method
In to apply_prompting method
out of apply_prompting method
IN to check_crisis method
out of check_crisis method
In to analyze_and_store_response method
Out of analyze_and_store_response method
In to clean_response method
Out of clean_response method
out of generate_response method


'Your appointment details have been successfully stored.'

In [ ]:
r , chat = chatbot.generate_response("thank you")
r

In to generate_response method
In to apply_prompting method
out of apply_prompting method
IN to check_crisis method
out of check_crisis method
In to clean_response method
Out of clean_response method
out of generate_response method


'““If you are worried about sleeping, then you should get up. If you are concerned about breathing, then you should be taking a deep breath. You can be mindful of your breath, that you can try to breathe in, relax your body, relax, and breathe in your mind. Breathe should allow your mind to your breath out. Then, your breath in, relax and relax your breathing in and let your breathing in.'

In [ ]:
r , chat = chatbot.generate_response("Suicide")
r

In to generate_response method
In to apply_prompting method
out of apply_prompting method
IN to check_crisis method
out of check_crisis method
IN to emergency_reply method
out of emergency_reply method
In to clean_response method
Out of clean_response method
out of generate_response method


'Im really sorry youre feeling this way. Your safety is important. Im here to provide information, but I cant offer therapy. I can help you get specialists who can help you. what is your location within following locations: - New York - Chicago - Los Angeles'

In [ ]:
r , chat = chatbot.generate_response("Gaza")
r

In to generate_response method
In to apply_prompting method
out of apply_prompting method
IN to check_crisis method
out of check_crisis method
In to experts_extraction method
IN to extract_location method
out of extract_location method
IN to find_experts_by_location method
out of find_experts_by_location method
out of experts_extraction method
In to clean_response method
Out of clean_response method
out of generate_response method


'Sorry, we couldnt find any specialists in your location (Gaza). We understand your desire to receive support. In this case, you can try one of the following options: 1. Contact local emergency services if you have an urgent situation. 2. Search for specialists online or through mental health platforms available in your area. 3. If you need immediate assistance, you can reach out to helplines available 24/7.If you need further assistance or suggestions, Im here to listen and support you.'

In [ ]:
r , chat = chatbot.generate_response("ok ,thank you")
r

In to generate_response method
In to apply_prompting method
out of apply_prompting method
IN to check_crisis method
out of check_crisis method
In to clean_response method
Out of clean_response method
out of generate_response method


'...Im a nurse and Im here to work with you. I have been diagnosed with insomnia and Ive been having a lot of anxiety and I have been having nightmares and Ive been dealing with it for a lot of nightmares. Im not good night before this is that Im a lot of nightmares and nightmares and Im scared of things have a lot of nightmares and Im not going through my body is not sleeping with these things. Ive been feeling like I have been having'

# Interface

In [ ]:
def chat_interface(message, history):
    """Gradio chat interface function that shows user and bot messages"""
    try:
        # Generate response
        response = chatbot.generate_response(message)
        cleaned_response = chatbot.clean_response(response)

        # Add some safety checks for mental health context
        if any(word in message.lower() for word in ['suicide', 'kill myself', 'end it all', 'hurt myself']):
            cleaned_response = ("⚠️ I'm really concerned about what you're sharing. Please reach out to a crisis helpline immediately:\n\n"
                              "🆘 National Suicide Prevention Lifeline: **988**\n"
                              "📱 Crisis Text Line: Text **HOME** to **741741**\n\n"
                              "You don't have to go through this alone. " + cleaned_response)

        # Append the conversation to history
        history.append([message, cleaned_response])
        return history, ""

    except Exception as e:
        error_response = "I'm here to support you. Could you please rephrase your question?"
        history.append([message, error_response])
        return history, ""

In [ ]:
css = """
.gradio-container {
    max-width: 900px !important;
    margin: auto !important;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

/* Disclaimer styling */
.disclaimer {
    color: #2d3436;
    padding: 20px;
    border-radius: 12px;
    margin-bottom: 20px;
    border: 1px solid #dfe6e9;
}

.disclaimer h2 {
    margin-top: 0;
    color: #636e72;
}

.disclaimer ul {
    margin-bottom: 0;
}

/* Chat message styling */
.message-row {
    margin: 8px 0;
}

.user-message {
    color: #2d3436;
    padding: 12px 16px;
    border-radius: 18px 18px 4px 18px;
    margin-left: 20%;
    margin-right: 5px;
    border: 1px solid #dfe6e9;
}

.bot-message {
    color: #2d3436;
    padding: 12px 16px;
    border-radius: 18px 18px 18px 4px;
    margin-right: 20%;
    margin-left: 5px;
    border-left: 4px solid #636e72;
}

/* Input area styling */
.input-area {
    padding: 10px;
    display: flex;
    gap: 8px;
    align-items: center;
}

/* Send button styling */
button.send-btn {
    background-color: #4a90e2;
    color: white;
    border-radius: 25px;
    font-weight: bold;
    align-items: center;
    justify-content: center;
}

/* Clear button */
.clear-btn {
    border: 1px solid #dfe6e9;
    color: #636e72;
    padding: 8px 16px;
    border-radius: 25px;
    font-weight: bold;
    background: none;
    transition: all 0.2s ease;
}

.clear-btn:hover {
    background: #f0f0f0;
}

/* Footer styling */
.footer-info {
    text-align: center;
    margin-top: 20px;
    color: #636e72;
    border: 1px solid #dfe6e9;
    padding: 15px;
    border-radius: 12px;
    font-style: italic;
    font-size: 0.9em;
}
"""
# Create Gradio interface with enhanced chat display
with gr.Blocks(css=css, title="Mental Health Counseling Chatbot") as demo:
    gr.HTML("""
    <div class="disclaimer">
        <h2>🌟 Mental Health Support Chatbot</h2>
        <p><strong>Note:</strong> This chatbot is designed to provide supportive conversations and general guidance.
        It is NOT a replacement for professional mental health care. If you're experiencing a mental health crisis,
        please contact a healthcare professional or crisis helpline immediately.</p>
    </div>
    """)

    with gr.Row():
        with gr.Column():
            chatbot_interface = gr.Chatbot(
                value=[],
                elem_id="chatbot",
                bubble_full_width=False,
                height=500,
                label="💬 Mental Health Support Chat",
                show_label=True,
                avatar_images=["👤", "🤖"],  # User and bot avatars
                show_share_button=False
            )

    with gr.Row(elem_classes="input-area"):
        with gr.Column(scale=4):
            msg = gr.Textbox(
                placeholder="💭 Share what's on your mind... I'm here to listen and support you.",
                label="Your Message",
                lines=4,
                max_lines=5,
                show_label=False,
                container=False
            )
        with gr.Column(scale=1, min_width=100):
            send_btn = gr.Button("Send",elem_classes="send-btn",size="lg")
            clear = gr.Button("Clear", variant="secondary", elem_classes="clear-btn")

    # Example questions for quick testing
    with gr.Row():
        gr.Examples(
            examples=[
                ["I've been feeling really anxious lately and can't seem to shake it off."],
                ["I don't feel motivated to do anything anymore."],
                ["I feel like I'm not good enough compared to others."],
                ["I can't fall asleep because my mind won't stop racing."],
                ["I'm having problems communicating with my partner."],
                ["What are some healthy ways to deal with stress?"],
                ["I feel stuck in my current situation and don't know how to change."],
            ],
            inputs=msg,
            label="💡 Try these example questions:"
        )

    # Set up the chat functionality for both button click and enter key
    msg.submit(
        chat_interface,
        [msg, chatbot_interface],
        [chatbot_interface, msg]
    )

    send_btn.click(
        chat_interface,
        [msg, chatbot_interface],
        [chatbot_interface, msg]
    )

    clear.click(lambda: ([], ""), None, [chatbot_interface, msg], queue=False)

    gr.HTML("""
    <div class="footer-info">
        <p><strong>🧠 AI-Powered Mental Health Support</strong></p>
        <p>This chatbot uses a fine-tuned DialoGPT model trained on mental health counseling conversations with advanced few-shot prompting.</p>
        <p><em>Remember: You are not alone, and it's okay to seek help. Your mental health matters. 💙</em></p>
        <p style="font-size: 0.9em; margin-top: 10px;">
            <strong>Features:</strong> Crisis detection • Empathetic responses • Professional guidance • 24/7 availability
        </p>
    </div>
    """)

print("Launching Gradio interface...")
print("The chatbot is ready! You can now interact with the mental health counseling assistant.")
print("\nNote: This model has been fine-tuned on mental health counseling conversations")
print("and uses few-shot prompting to provide more empathetic and helpful responses.")

# Launch the interface
demo.launch(
    share=True,  # Creates a public link
    debug=False,
    server_name="0.0.0.0"
)

/tmp/ipython-input-1933739395.py:107: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(
/tmp/ipython-input-1933739395.py:107: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot_interface = gr.Chatbot(


Launching Gradio interface...
The chatbot is ready! You can now interact with the mental health counseling assistant.

Note: This model has been fine-tuned on mental health counseling conversations
and uses few-shot prompting to provide more empathetic and helpful responses.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9e93f50f14dbace432.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
